In [1]:
import pandas as pd
import numpy as np
import jupyter_black
import nfl_data_py as nfl
from typing import Literal, Tuple

# from src.modelling.metrics import success_rate_lambda
# from visualisation.plots import plot_team_scatter

jupyter_black.load()
pd.set_option("display.max_columns", None)

In [2]:
pbp = nfl.import_pbp_data(range(2022, 2025))

2022 done.
2023 done.
2024 done.
Downcasting floats.


# Rushing EPA

In [17]:
pbp_rp = pbp.query('play_type=="pass" or play_type=="run"')
len(pbp_rp)

106080

In [19]:
# filter for neutral game states
neutral_wp_threshold = 0.025

pbp_rp_neu = pbp_rp.query(
    f"wp>={neutral_wp_threshold} and def_wp>={neutral_wp_threshold}"
)
len(pbp_rp_neu)

94905

In [ ]:
pwk = nfl.import_weekly_data(years=[2024])
len(pwk)

Downcasting floats.


5597

In [31]:
pwk.position_group.value_counts()

WR      2238
RB      1480
TE      1137
QB       697
DB        18
SPEC      14
OL         9
LB         3
DL         1
Name: position_group, dtype: int64

In [33]:
print(pwk.sample(4).to_markdown())
# pwk.sample(4)

|      | player_id   | player_name   | player_display_name   | position   | position_group   | headshot_url                                                                      | recent_team   |   season |   week | season_type   | opponent_team   |   completions |   attempts |   passing_yards |   passing_tds |   interceptions |   sacks |   sack_yards |   sack_fumbles |   sack_fumbles_lost |   passing_air_yards |   passing_yards_after_catch |   passing_first_downs |   passing_epa |   passing_2pt_conversions |   pacr |   dakota |   carries |   rushing_yards |   rushing_tds |   rushing_fumbles |   rushing_fumbles_lost |   rushing_first_downs |   rushing_epa |   rushing_2pt_conversions |   receptions |   targets |   receiving_yards |   receiving_tds |   receiving_fumbles |   receiving_fumbles_lost |   receiving_air_yards |   receiving_yards_after_catch |   receiving_first_downs |   receiving_epa |   receiving_2pt_conversions |     racr |   target_share |   air_yards_share |      wopr |   s

In [47]:
qb_cols = [
    "player_id",
    "player_name",
    "player_display_name",
    "recent_team",
    "season",
    "week",
    "opponent_team",
    "completions",
    "attempts",
    "passing_yards",
    "passing_tds",
    "interceptions",
    "sacks",
    "sack_yards",
    "sack_fumbles",
    "sack_fumbles_lost",
    "passing_air_yards",
    "passing_yards_after_catch",
    "passing_first_downs",
    "passing_epa",
    "passing_2pt_conversions",
    "pacr",
    "dakota",
    "carries",
    "rushing_yards",
    "rushing_tds",
    "rushing_fumbles",
    "rushing_fumbles_lost",
    "rushing_first_downs",
    "rushing_epa",
    "rushing_2pt_conversions",
    "fantasy_points",
]

In [48]:
pwk_qb = pwk.query("position=='QB'").query("season_type=='REG'")[qb_cols].copy()
len(pwk_qb)

664

In [49]:
pwk_qb.sample(4)

,player_id,player_name,player_display_name,recent_team,season,week,opponent_team,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,fantasy_points
36,00-0026498,M.Stafford,Matthew Stafford,LA,2024,9,SEA,25,44,298.0,2,1.0,0.0,-0.0,0,0,436.0,108.0,13.0,3.873419,0,0.683486,0.096348,1,0.0,0,1.0,0.0,0.0,-0.969130,0,17.920000
5179,00-0039732,B.Nix,Bo Nix,DEN,2024,11,ATL,28,33,307.0,4,0.0,1.0,12.0,0,0,139.0,199.0,14.0,22.721409,0,2.208633,0.308354,2,5.0,0,0.0,0.0,0.0,-0.622357,0,28.780001
3044,00-0036972,M.Jones,Mac Jones,JAX,2024,17,TEN,15,22,174.0,2,0.0,2.0,1.0,0,0,124.0,101.0,9.0,7.608073,0,1.403226,0.196548,6,22.0,0,0.0,0.0,4.0,3.823416,0,17.160000
5550,00-0039918,C.Williams,Caleb Williams,CHI,2024,5,CAR,20,29,304.0,2,0.0,1.0,8.0,0,0,208.0,182.0,12.0,15.089914,0,1.461538,0.217157,5,34.0,0,0.0,0.0,3.0,3.099502,0,23.559999


In [ ]:
week = 13

In [12]:
off_epa_df = (
    pbp_r_neu.groupby(["season", "week", "posteam"])
    .agg({"epa": "mean"})
    .sort_values(by="epa", ascending=False)
)
# off_epa_df["epa_rank"] = off_epa_df["epa"].rank(ascending=False).astype(int)
off_epa_df["epa_percentile"] = off_epa_df["epa"].rank(pct=True).round(2) * 10

In [13]:
off_epa_df.xs(2024, level="season").xs(2, level="week")

,epa,epa_percentile
posteam,,
NO,1.598735,10.0
ARI,1.598454,10.0
BUF,0.602363,9.5
GB,0.540034,9.4
ATL,0.293003,8.1
NYG,0.220160,7.4
SEA,0.195122,7.1
DAL,0.179687,7.0
LAC,0.165951,6.9


In [ ]:
pbp_r_neu.groupby(["season", "posteam"]).agg({"epa": "mean"}).sort_values(
    by="epa", ascending=False
).rename(columns={"epa": "epa_per_rush"})

In [ ]:
success_rate_lambda = lambda x: 1 if x > 0 else 0

In [ ]:
def calculate_epa_metrics(
    data: pd.DataFrame, team: Literal["posteam", "defteam"] = "posteam"
) -> pd.DataFrame:
    sort_ascending = team == "defteam"
    df = data.copy()
    df["success"] = df["epa"].apply(success_rate_lambda)
    epa_df = (
        df.groupby(team)
        .agg({team: "count", "epa": "mean", "success": "mean"})
        .sort_values(by="epa", ascending=False)
        .rename(columns={team: "n", "success": "success_rate"})
    )
    for col in ["epa", "success_rate"]:
        epa_df[f"{col}_rank"] = epa_df[col].rank(ascending=sort_ascending).astype(int)
        epa_df[f"{col}_percentile"] = (
            epa_df[col].rank(ascending=(not sort_ascending), pct=True).round(2) * 10
        )
    epa_df["epa"] = epa_df["epa"].round(3)
    epa_df["success_rate"] = epa_df["success_rate"].round(2)
    col_list = list(epa_df.columns)
    col_list.remove("n")
    col_list = list(sorted(col_list))
    col_list.insert(0, "n")
    return epa_df[col_list].sort_values("epa_rank")


def dual_epa_metrics(data: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    return calculate_epa_metrics(data), calculate_epa_metrics(data, "defteam")

In [ ]:
off_rush_epa_df, def_rush_epa_df = dual_epa_metrics(pbp_r_neu.query("season==2023"))

In [ ]:
off_rush_epa_df

In [ ]:
plot_team_scatter(off_rush_epa_df, "success_rate", "epa", alpha=0.9)

In [ ]:
def_rush_epa_df

In [ ]:
plot_team_scatter(def_rush_epa_df, "success_rate", "epa", flip_def=True, alpha=0.9)